In [126]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
seed(1)

In [127]:
#operator definitionis
N = 2
si = qeye(2)
sx = 0.5*sigmax()
sy = 0.5*sigmay()
sz = 0.5*sigmaz()

In [128]:
#make list of tensor products
sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    op_list = []
    for m in range(N):
        op_list.append(si)

    op_list[n] = sx
    sx_list.append(tensor(op_list))

    op_list[n] = sy
    sy_list.append(tensor(op_list))

    op_list[n] = sz
    sz_list.append(tensor(op_list))


In [129]:
#Construct Hamiltonian
B = 0
A0 = 1
H = B*sz_list[0] 
for n in range(N-1):
    H += A0*sz_list[0]*sz_list[n+1] + A0*sx_list[0]*sx_list[n+1] + A0*sy_list[0]*sy_list[n+1]
print(H)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


In [130]:
#Find Groundstate
H.groundstate()

(-0.7500000000000001,
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.        ]
  [ 0.70710678]
  [-0.70710678]
  [ 0.        ]])

In [131]:
#Make basis
Sbasis = []
for i in range(N):
    for j in range(N):
        Sbasis.append(tensor([basis(N,i),basis(N,j)]))
print(Sbasis)


[Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]]


In [132]:
#get sz values for basis states
sz = np.zeros((2**N,N), dtype = complex)
a = [[1 for j in range(N)] for i in range(2**N)]
for i in range(2**N):
    for j in range(N):
        #matrix element <bra|Sz|ket>
        sz[i][j] = sz_list[j].matrix_element(Sbasis[i],Sbasis[i])

print(sz)

[[ 0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]
 [-0.5+0.j -0.5+0.j]]


In [133]:
#Define RBM Parameters
M = 2
alp = M/N

a = rand(N)
b = rand(M)
W = rand(M,N)

In [134]:
#Function to give RBM wavefuntion
def RBM_ansatz(a, b, W, Sbasis, N, M,sz):
    expTerm = np.zeros(2**N, dtype = complex)
    coshTerm = np.zeros((M,2**N), dtype = complex)
    hidProduct = np.zeros(2**N, dtype = complex)
    psiMValues = np.zeros(2**N, dtype = complex)
    psiM = 0*Sbasis[0]

    for i in range(2**N):
        for m in range(M):
            coshTerm[m][i] = np.cosh(np.dot(W[m],sz[i]) + b[m])

    hidProduct = 2*np.prod(coshTerm, axis = 0)   
    
    for i in range(2**N):
        expTerm[i] = np.exp(np.dot(a,sz[i]))
        psiMValues[i] = expTerm[i]*hidProduct[i]
        psiM += psiMValues[i]*Sbasis[i]
        
    return psiM


In [135]:
#Variation Energy Definition
def varEnergy(a, b, W, Sbasis, N, M,sz, H):
    psiM = RBM_ansatz(a, b, W, Sbasis, N, M, sz)
    E = expect(H,psiM)
    norm = psiM.norm()**2
    Enorm = E/norm
    return Enorm

E = varEnergy(a, b, W, Sbasis, N, M,sz, H)
print(E)

0.24021684121030115


In [141]:
#Gradient Descent 
def updateParameters(a, b, W, learning_rate):
    #Find partials
    a_partials = optimize.approx_fprime(a,varEnergy,eps,b,W,Sbasis, N, M,sz, H)
    b_partials = optimize.approx_fprime(b,varEnergy,eps,a,W,Sbasis, N, M,sz, H)
print(b_partials)
    for
    return a, b, W




In [142]:
#Ground State Wavefunction



In [183]:
#test function 
eps = 0.5

a_eps = np.zeros((N,N))
for i in range(N):
    a_eps[i] = np.copy(a)
    a_eps[i][i] += eps


    


print(a)
print(a_eps)

grad = (varEnergy(a_esp, b, W, Sbasis, N, M,sz, H)-varEnergy(a, b, W, Sbasis, N, M,sz, H))/eps
print(grad)

[2.917022   0.72032449]
[[3.417022   0.72032449]
 [2.917022   1.22032449]]
-0.01288211857324284


In [192]:

a_partials = optimize.approx_fprime(a,varEnergy,eps,b,W,Sbasis, N, M,sz, H)
print(a_partials)

b_partials = optimize.approx_fprime(b,varEnergy,eps,a,W,Sbasis, N, M,sz, H)
print(b_partials)

W_partials = optimize.approx_fprime(W,varEnergy,eps,a,b,Sbasis, N, M,sz, H)
print(W_partials)


[-0.01288212  0.07851836]
[ 0.02036484 -0.08301088]


TypeError: only length-1 arrays can be converted to Python scalars